In [ ]:
from utils import *
from diffusers import StableDiffusionPipeline
import json
import torch

In [25]:
# Load the model

device = "cuda"

model_path = "/root/annotated_textual_inversion/model/sd-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe = pipe.to(device)
pipe.safety_checker = lambda images, clip_input: (images, False)

/root/annotated_textual_inversion/repositories/transformers/src/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [26]:
# Load embedding 1

token_1 = "<bird>"
number = 2
weight_dir="/root/annotated_textual_inversion/embeds/bird-MTI-2-1200" # MTI-2-1200; ATI-1200; TI-2000
name="learned_embeds"

# for MTI
token_1 = load_MTI_embeds(pipe, model_path, token_1, number, weight_dir, name)

# for TI
# pipe.load_textual_inversion(model_path, token_1, weight_name=os.path.join(weight_dir, f"{name}.bin"))

In [27]:
# Load embedding 2

token_2 = "<flower>"
number = 2
weight_dir="/root/annotated_textual_inversion/embeds/flower-MTI-2-1200" # MTI-2-1200; ATI-1200; TI-2000
name="learned_embeds"

# for MTI
token_2 = load_MTI_embeds(pipe, model_path, token_2, number, weight_dir, name)

# for TI
# pipe.load_textual_inversion(model_path, token_2, weight_name=os.path.join(weight_dir, f"{name}.bin"))

In [28]:
# set parameters

# a {token_2} robot in {token_1} style; {token_1} style, a {token_2} robot; a painting in {token_1} style with a {token_2} robot
prompt = f"a {token_1} bird next to {token_2} flower" 
output_dir = f"/root/annotated_textual_inversion/output/samples"
num_samples = 10
step = 50
guidance_scale = 7.5
generator = torch.Generator(device="cuda").manual_seed(0)

In [29]:
# generate samples

pipe.set_progress_bar_config(disable=True)
os.makedirs(output_dir, exist_ok=True)

print(f"generating samples ...")
for i in trange(num_samples):
    sample = pipe(prompt=prompt, num_inference_steps=step, guidance_scale=guidance_scale, generator=generator).images
    sample[0].save(f"{output_dir}/sample-{i}.png")

generating samples ...


100%|██████████| 10/10 [00:37<00:00,  3.78s/it]
